## Introduction to the GLM

Now that the data is clean we can perform analysis. Resting state analysis is completed in notebook 6. This is a review of the general linear model using data that was collected during a breathholding task. 

### Step 1: Ensure you have data from the breath-hold task downloaded to your local machine. 
    
    #TODO: link to download, currently aws but Erin may move to datalad
    
### Step 2: We need to figure out the stimulus for the breath holding task. 

The description of this task is contained in `data/ds000030/task-bht_bold.json`. Read through this folder and determine if you can create an array of timings where the subject was and was not holding their breath.

In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

TR = 2
scan_length = 166

act_length = 13.5
rest_length = 13.5
n_blocks = scan_length/(act_length + rest_length)
onsets = np.linspace(13.5, n_blocks*(act_length + rest_length), n_blocks)

volumes = np.linspace(1, scan_length-1, scan_length/TR) 

print(onsets, volumes)


[ 13.5  44.   74.5 105.  135.5 166. ] [  1.   3.   5.   7.   9.  11.  13.  15.  17.  19.  21.  23.  25.  27.
  29.  31.  33.  35.  37.  39.  41.  43.  45.  47.  49.  51.  53.  55.
  57.  59.  61.  63.  65.  67.  69.  71.  73.  75.  77.  79.  81.  83.
  85.  87.  89.  91.  93.  95.  97.  99. 101. 103. 105. 107. 109. 111.
 113. 115. 117. 119. 121. 123. 125. 127. 129. 131. 133. 135. 137. 139.
 141. 143. 145. 147. 149. 151. 153. 155. 157. 159. 161. 163. 165.]


/Users/oliviastanley/miniconda3/envs/conpcourse/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  # This is added back by InteractiveShellApp.init_path()
/Users/oliviastanley/miniconda3/envs/conpcourse/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  del sys.path[0]


### Step 3: Now that we know what stimulus was used we need to convolve it with a hemodynamic response function. 

This can be done using using the nitime.fmri.hrf function. 

    #TODO: Full guide

### Step 4: Now we can fit our model of what task was acquired to our input data

This can be done using the permuted ols function from nilearn. We can also include motion confounds as before simulataneously in order to remove them during fitting opposed to cleaning the data. This is the standard for task based data.

### Step 5: Visualizing fit results

Now to visualize the results of our fit. Lets look at a plot of the most active voxel and a show an image of estimates beta weights.

We can also look at the residuals across space and time. These can show issues such as motion or physiology contamination.

### Step 6: Improving our fit with pre-whitening

The OLS fit we ran before did not take into account the covariance of our data. 

### Step 7: Wrapping up and Final Thoughts